In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [ ]:
# read in data - ouytput of GWR and ML models
df = pd.read_csv('minimize_data.csv')


# ignore the blocks that can't be allocated trees
df = df[df["number_potential_trees"]>0]
df = df[df["teffect_cover"]<=-0.8840332]

#df = df.head(5000)

In [ ]:
print(len(df))
df.head()

# Temperature Scenario # 1

In [ ]:
# For city blocks i in 1:40,000:
# Distribute trees t in 1:75,000:
# Such that Mean Temperature across all blocks is minimized: 
# (Constraint) And where ti (trees distributed to block i) must be < available tree space in block i:
 
# Temperaturei = Ai * (estimated tree cover + ti) 
 
# Ai ; Bi ; Ci  have all been calculated at the city block level via GWR


trees_to_distribute = 75000 # will be 75,000 in final model based on Chicago policy

blocks = np.array(df["geoid10"]) # final set of equations will have 40k+ blocks
xintercept = np.array(df["xintercept_cover"]).astype("float32")
tcover_effect = np.array(df["teffect_cover"]).astype("float32") # comes from GWR
cluster_tree_constraints = np.array(df["number_potential_trees"]).astype("int32") # calculated based on available vegetation space w/o trees currently

current_trees = np.array(df["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover


tree_factor= np.array(df["area_added_by_tree"]).astype("float32") # This is the pct of cover increased for each additional tree (per block)

# calculate baseline temp based on GWR
current_mean_temp = xintercept + tcover_effect*(current_trees)

In [ ]:
%%time
# Temperaturei = Ai * (estimated tree cover + ti) 
def eq( p ):
    minimize_array = p
    function_array = xintercept + tcover_effect*(current_trees + (minimize_array*tree_factor))
    final_temp = np.mean(function_array)
    return (final_temp)

# Set boundary values between 0 and max # of trees that can be added
bnds = [(0, x) for x in cluster_tree_constraints]
bnds = tuple(bnds)

# Force it to distribute exactly 75,000 trees
def con1(p):
    return sum(p) - trees_to_distribute

cons= ( { 'type' : 'eq', 'fun': con1} )


final=minimize( eq, tuple(np.random.randint(0,100,5000).astype("int32")),  bounds=bnds, constraints=cons )

# CPU times: total: 44min 32s
# Wall time: 46min 38s

In [ ]:
# Looks to be working fine, will try on full dataset...
print(final)
print(np.sum(final["x"]))

# This seems to work; now just need to figure out how to scale this up when there are 40k blocks and 40k of each of the GWR tree effect values

print("Total # of trees allocated:", sum(final.x))
print(f"Cluster {blocks[3]} is allocated {np.round(final.x[3])}  trees")

print(np.round(final["x"]))
print(cluster_tree_constraints)

#   fun: 0.12623861589868213
#      jac: array([-2.79396772e-07, -1.93715096e-07, -3.93018126e-07, ...,
#        -6.37024641e-07, -6.91041350e-07, -1.15483999e-06])
#  message: 'Optimization terminated successfully'
#     nfev: 10003
#      nit: 2
#     njev: 2
#   status: 0
#  success: True
#        x: array([1.80240461e+01, 3.30240459e+01, 2.53117267e-21, ...,
#        0.00000000e+00, 0.00000000e+00, 1.72825665e-07])
# 74999.99999999983
# Total # of trees allocated: 74999.9999999998
# Cluster 170317402004007 is allocated 0.0  trees
# [18. 33.  0. ...  0.  0.  0.]
# [384 547 192 ...  76  15   7]

In [ ]:
# Save results
#allocated_trees=np.round(final["x"])
#df["allocated_trees"] = allocated_trees
# allocated_trees =  pd.read_csv('paper_allocated_trees_coveralone.csv')
# df["allocated_trees"] = np.array(allocated_trees["allocated_trees_coveralone"])
# df.to_csv(/paper_allocated_trees_coveralone.csv")

# Temperature Scenario # 2

In [ ]:
# read in data
df = pd.read_csv('minimize_data.csv')


# ignore the blocks that can't be allocated trees
df = df[df["number_potential_trees"]>0]
df = df[df["teffect_cover"]<=-0.8866490]


In [ ]:
# Temperature Scenario # 2
# For city blocks i in 1:40,000:
# Distribute trees t in 1:75,000:
# Such that Mean Temperature across all blocks is minimized: 
# (Constraint) And where ti (trees distributed to block i) must be < available tree space in block i:
 
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
 
# Ai ; Bi ; Ci  have all been calculated at the city block level via GWR

trees_to_distribute = 75000 # will be 75,000 in final model

blocks = np.array(df["geoid10"]) # final set of equations will have 40k+ blocks
theight_effect = np.array(df["heighteffect_coverxheight"]).astype("float32")
tinteraction_effect = np.array(df["interactioneffect_coverxheight"]).astype("float32") # comes from GWR

xintercept = np.array(df["xintercept_coverxheight"]).astype("float32")
tcover_effect = np.array(df["teffect_coverxheight"]).astype("float32") # comes from GWR
cluster_tree_constraints = np.array(df["number_potential_trees"]).astype("int32") # calculated based on available vegetation space w/o trees currently

current_trees = np.array(df["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover
current_height = np.array(df["avg_canopy_height_scaled"]).astype("float32") # from UNet, predicted tree cover

tree_factor= np.array(df["area_added_by_tree"]).astype("float32") # This is the pct of cover increased for each additional tree (per block)

In [ ]:
%%time
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
def eq( p ):
    minimize_array = p
    function_array = xintercept + (tcover_effect*(current_trees + (minimize_array*tree_factor))) + (theight_effect*current_height) + (tinteraction_effect*(current_trees + (minimize_array*tree_factor))*current_height)
    final_temp = np.mean(function_array)
    return (final_temp)

# Set boundary values between 0 and max # of trees that can be added
bnds = [(0, x) for x in cluster_tree_constraints]
bnds = tuple(bnds)

# Force it to distribute exactly 1,000 trees
def con1(p):
    return sum(p) - trees_to_distribute

cons= ( { 'type' : 'eq', 'fun': con1} )


final=minimize( eq, tuple(np.random.randint(0,100,4958).astype("int32")),  bounds=bnds, constraints=cons )

# CPU times: total: 43min 38s
# Wall time: 55min 5s

In [ ]:
# Looks to be working fine, will try on full dataset...
print(final)
print(np.sum(final["x"]))

# This seems to work; now just need to figure out how to scale this up when there are 40k blocks and 40k of each of the GWR tree effect values

print("Total # of trees allocated:", sum(final.x))
print(f"Cluster {blocks[3]} is allocated {np.round(final.x[3])}  trees")

print(np.round(final["x"]))
print(cluster_tree_constraints)

#    fun: 0.1326607622800871
#      jac: array([-3.40864062e-07, -2.79396772e-07, -4.33996320e-07, ...,
#        -7.35744834e-07, -6.50063157e-07, -6.14672899e-07])
#  message: 'Optimization terminated successfully'
#     nfev: 9919
#      nit: 2
#     njev: 2
#   status: 0
#  success: True
#        x: array([20.88863432, 25.88863421, 40.88863452, ..., 38.88863512,
#         0.        ,  0.        ])
# 75000.00000000017
# Total # of trees allocated: 75000.0000000001
# Cluster 170317402004007 is allocated 0.0  trees
# [21. 26. 41. ... 39.  0.  0.]
# [384 547 192 ...  76  15   7]

In [ ]:
# Save results
# allocated_trees=np.round(final["x"])

# df["allocated_trees"] = allocated_trees
# df.to_csv("paper_allocated_trees_coverheight.csv")

# # allocated_trees =  pd.read_csv('paper_allocated_trees_coverheight.csv')
# # df["allocated_trees"] = np.array(allocated_trees["allocated_trees_coverheight"])
# # df.to_csv("paper_allocated_trees_coverheight2.csv")

# Equality Scenario # 1

In [ ]:
# read in data
df = pd.read_csv('minimize_data.csv')


# ignore the blocks that can't be allocated trees
df = df[df["number_potential_trees"]>0]
df2=df.copy()
df2 = df2[df2["pct_tree_cover"]>0.22]
df = df[df["pct_tree_cover"]<=0.22]


In [ ]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [ ]:
# For city blocks i in 1:40,000:
# Distribute trees t in 1:75,000:
# Such that Gini Coefficient across all blocks is minimized: 
# (Constraint) And where ti (trees distributed to block i) must be < available tree space in block i:
 
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
 
# Ai ; Bi ; Ci  have all been calculated at the city block level via GWR

old_tree_distribution =np.array(df2["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover

trees_to_distribute = 75000 # will be 75,000 in final model

cluster_tree_constraints = np.array(df["number_potential_trees"]).astype("int32") # calculated based on available vegetation space w/o trees currently

current_trees = np.array(df["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover
current_height = np.array(df["avg_canopy_height_scaled"]).astype("float32") # from UNet, predicted tree cover

tree_factor= np.array(df["area_added_by_tree"]).astype("float32") # This is the pct of cover increased for each additional tree (per block)

In [ ]:
%%time
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
def eq( p ):
    minimize_array = p
    new_tree_distribution =  current_trees + (minimize_array*tree_factor)
    temp_gini =  gini(np.append(new_tree_distribution, old_tree_distribution))
    return (temp_gini)

# Set boundary values between 0 and max # of trees that can be added
bnds = [(0, x) for x in cluster_tree_constraints]
bnds = tuple(bnds)

# Force it to distribute exactly 1,000 trees
def con1(p):
    return sum(p) - trees_to_distribute

cons= ( { 'type' : 'eq', 'fun': con1} )


final=minimize( eq, tuple(np.random.randint(0,100,11577).astype("int32")),  bounds=bnds, constraints=cons )

In [ ]:
# Save
# print("Total # of trees allocated:", sum(final.x))
# print(np.round(final["x"]))
# print(cluster_tree_constraints)

# allocated_trees=np.round(final["x"])

# df["allocated_trees"] = allocated_trees
# df.to_csv("paper_allocated_trees_equity.csv")

# Equality Scenario # 2

In [ ]:
# read in data
df = pd.read_csv('D:/final_data/paper_data/minimize_data.csv')


# ignore the blocks that can't be allocated trees
df = df[df["number_potential_trees"]>0]
df2=df.copy()
df = df[df["pct_tree_cover"]<=0.22]
df = df[df["avg_canopy_height_scaled"]<=.3]

df2 = df2[~df2.index.isin(df.index)]

In [ ]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [ ]:
# For city blocks i in 1:40,000:
# Distribute trees t in 1:75,000:
# Such that Gini Coefficient across all blocks is minimized: 
# (Constraint) And where ti (trees distributed to block i) must be < available tree space in block i:
 
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
 
# Ai ; Bi ; Ci  have all been calculated at the city block level via GWR

old_tree_distribution =np.array(df2["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover
old_height_distribution =np.array(df2["avg_canopy_height_scaled"]).astype("float32") # from UNet, predicted tree cover

trees_to_distribute = 75000 # will be 75,000 in final model

cluster_tree_constraints = np.array(df["number_potential_trees"]).astype("int32") # calculated based on available vegetation space w/o trees currently

current_trees = np.array(df["pct_tree_cover"]).astype("float32") # from UNet, predicted tree cover
current_height = np.array(df["avg_canopy_height_scaled"]).astype("float32") # from UNet, predicted tree cover

tree_factor= np.array(df["area_added_by_tree"]).astype("float32") # This is the pct of cover increased for each additional tree (per block)
height_factor=3.800962 # 0.2753555 feet increase in height for a 1% increase in pct_tree cover

In [ ]:
%%time
# Temperaturei = Ai * (estimated tree cover + ti) +  Bi * (estimated canopy height + ti) + Ci*(estimated tree cover + ti)*(estimated canopy height + ti)
def eq( p ):
    minimize_array = p
    new_tree_distribution =  current_trees + (minimize_array*tree_factor)
    temp_gini1 =  gini(np.append(new_tree_distribution, old_tree_distribution))
    
    # add in height as a factor which gives a bonus based on pct_tree_cover gained
    new_height_distribution =  (current_height + ((minimize_array*tree_factor)*height_factor))
    temp_gini2 =  gini(np.append(new_height_distribution, old_height_distribution))
    
    # minimize the sum of both ginis
    temp_gini = temp_gini1 + temp_gini2
    
    return (temp_gini)

# Set boundary values between 0 and max # of trees that can be added
bnds = [(0, x) for x in cluster_tree_constraints]
bnds = tuple(bnds)

# Force it to distribute exactly 1,000 trees
def con1(p):
    return sum(p) - trees_to_distribute

cons= ( { 'type' : 'eq', 'fun': con1} )


final=minimize( eq, tuple(np.random.randint(0,100,6910).astype("int32")),  bounds=bnds, constraints=cons )

In [ ]:
# Save
# print(final)
# print(np.sum(final["x"]))
# print("Total # of trees allocated:", sum(final.x))
# print(np.round(final["x"]))
# print(cluster_tree_constraints)

# allocated_trees=np.round(final["x"])

# df["allocated_trees"] = allocated_trees
# df.to_csv("paper_allocated_trees_equityheight.csv")